In [ ]:
from statsbombpy import sb
import pandas as pd
from pandas import json_normalize
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as path_effects

# We'll only use a vertical pitch for this tutorial
from mplsoccer import VerticalPitch, Sbopen

# Get competitions
comp = sb.competitions()

In [ ]:
comp.head(1)

In [ ]:
matches= sb.matches(9,281)

In [ ]:
matches.head(1)

In [ ]:
events= sb.events(3895302)

In [ ]:
# ÷unique values in type colun

unique_types = events['type'].unique()
print(unique_types)

In [ ]:
# row ehere type is "Foul Won"
foul_won = events[events['type'] == 'Foul Committed']


In [ ]:
foul_won.to_csv('foul_won.csv', index=False)

In [ ]:
events.columns

In [ ]:
sb.lineups(3895302)

In [ ]:
a=sb.matches(competition_id=55, season_id=282)

In [ ]:
a

In [ ]:
events = sb.events(match_id=3930161)
events["type"].unique()

In [ ]:
events_type = events['type'].unique()
print(events_type)
# df where type is 'Own Goal For' 'Own Goal Against'
own_goals = events[events['type'].isin(['Own Goal For', 'Own Goal Against'])]
print(own_goals.to_dict(orient='records'))

In [ ]:
# postion unique values
unique_positions = events['position'].unique()
# print(unique_positions)

# ITERATE OVERL ALL MATCHES AND GET ALL POSSIBLE POSITIONS
all_positions = set()
eventss=[]
for match_id in a['match_id']:
    events = sb.events(match_id)
    eventss.extend(list(events["type"].unique()))
    eventss=set(eventss)
    all_positions.update(events['position'].dropna().unique())
# Convert to a sorted list
all_positions = sorted(all_positions)
print(all_positions)

In [ ]:
events = sb.events(match_id=3930167)
events.to_csv('events.csv', index=False)

In [ ]:
# gt all formations in matches in the competition
formations = set()
match_ids=[]
for match_id in a['match_id']:
    match_info = a[a['match_id'] == match_id].iloc[0]
    home_team = match_info['home_team']
    away_team = match_info['away_team']
    events = sb.events(match_id)
    f= events["tactics"].iloc[0]['formation']
    if f not in formations:
        formations.add(f)
        match_ids.append((match_id,home_team, away_team, f))



In [ ]:
match_ids

In [ ]:
formations, len(formations)

In [ ]:
parser = Sbopen()
event, related, freeze, tactics = parser.event(69249)

In [ ]:
 event['tactics_formation'].iloc[0]

In [ ]:
[0] * 11

In [ ]:
events[events['type'] == 'Substitution']
# print folowing columnsevents.columns
# type, substitution_replacement
substitutions = events[events['type'] == 'Substitution'][['player','type', 'substitution_replacement']]
print(substitutions)

In [ ]:
# Mateo Retegui,Gianluca Scamacca
players_of_interest = ['Mateo Retegui', 'Gianluca Scamacca']
events_of_interest = events[events['player'].isin(players_of_interest)]
columns_of_interest = ['player', 'type', 'pass_recipient', 'position', 'substitution_replacement']


In [ ]:
pass_outcomes = events['pass_outcome'].unique()
print(pass_outcomes)

In [ ]:
events_of_interest.to_dict()

In [ ]:
events_of_interest.to_csv('events_of_interest.csv', index=False)

In [ ]:
# row in a where match id is 3895302
match_row = a[a['match_id'] == 3930167]

In [ ]:
match_row.to_dict(orient='records')


In [ ]:
sb.lineups(3930167)["Albania"]

In [ ]:
events["tactics"].iloc[0]

In [ ]:
starting_xi_event = events.loc[((events['type'] == 'Starting XI') &
                               (events['team'] == 'Croatia')), ['id', 'tactics']]

In [ ]:
print(starting_xi_event)

In [ ]:
parser = Sbopen()
event, related, freeze, tactics = parser.event(3930167)

In [ ]:
related

In [ ]:
sb.lineups(3930167)

In [ ]:
parser = Sbopen()
event, related, freeze, tactics = parser.event(69249)
# starting players from Barcelona
starting_xi_event = event.loc[((event['type_name'] == 'Starting XI') &
                               (event['team_name'] == 'Barcelona')), ['id', 'tactics_formation']]
# joining on the team name and formation to the lineup
starting_xi = tactics.merge(starting_xi_event, on='id')
# replace player names with the shorter version
player_short_names = {'Víctor Valdés Arribas': 'Víctor Valdés',
                      'Daniel Alves da Silva': 'Dani Alves',
                      'Gerard Piqué Bernabéu': 'Gerard Piqué',
                      'Carles Puyol i Saforcada': 'Carles Puyol',
                      'Eric-Sylvain Bilal Abidal': 'Eric Abidal',
                      'Gnégnéri Yaya Touré': 'Yaya Touré',
                      'Andrés Iniesta Luján': 'Andrés Iniesta',
                      'Xavier Hernández Creus': 'Xavier Hernández',
                      'Lionel Andrés Messi Cuccittini': 'Lionel Messi',
                      'Thierry Henry': 'Thierry Henry',
                      "Samuel Eto''o Fils": "Samuel Eto'o"}
starting_xi['player_name'] = starting_xi['player_name'].replace(player_short_names)
# filter only succesful ball receipts from the starting XI
event = event.loc[((event['type_name'] == 'Ball Receipt') &
                   (event['outcome_name'].isnull()) &
                   (event['player_id'].isin(starting_xi['player_id']))
                   ), ['player_id', 'x', 'y']]
# merge on the starting positions to the events
event = event.merge(starting_xi, on='player_id')
formation = event['tactics_formation'].iloc[0]

In [ ]:
type(formation)

In [ ]:
from mplsoccer import Pitch
pitch = Pitch()
print(pitch.formations)

In [ ]:
def get_formation_offsets(formation: str) -> list:
    try:
        # Convert the string formation (e.g., '4231') into a list of integers
        lines = [int(x) for x in formation]
        if sum(lines) != 10:
            raise ValueError("Formation must sum to 10 outfield players.")

        offsets = [0]
        for num_players in lines:
            if num_players == 1:
                offsets.append(0)
            else:
                # spread players evenly with spacing of 6 units
                width = (num_players - 1) * 6
                line_offsets = list(range(-width // 2, width // 2 + 1, 6))
                if len(line_offsets) > num_players:  # remove extra if over
                    line_offsets = line_offsets[:num_players]
                offsets.extend(line_offsets)

        return offsets
    except ValueError as e:
        print(f"Error parsing formation '{formation}': {e}")
        return [0 for i in range(10)]  # Default to a flat line if parsing fails
xoffset =   get_formation_offsets("4321")
xoffset

In [ ]:
event.columns

In [8]:
from statsbombpy import sb
a=sb.competition_events(
        country='Europe',
        division='UEFA Euro',
        season='2024',
        gender="male"
    )



/opt/miniconda3/envs/viz/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/miniconda3/envs/viz/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/miniconda3/envs/viz/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/miniconda3/envs/viz/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/miniconda3/envs/viz/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  warnings.warn(
/opt/miniconda3/envs/viz/lib/python3.10/site-packages/statsbombpy/api_client.py:21: NoAuthWarning: credentials were not supplied. open data access only
  w

In [9]:
# a.columns
# list all columns which have outcome in it
outcome_columns = [col for col in a.columns if 'outcome' in col.lower()]
print(outcome_columns)

['ball_receipt_outcome', 'dribble_outcome', 'duel_outcome', 'goalkeeper_outcome', 'interception_outcome', 'pass_outcome', 'shot_outcome', 'substitution_outcome']


In [10]:
# list column name and unique values of it
for col in ["dribble_outcome", "pass_outcome", "shot_outcome", "duel_outcome","interception_outcome"]:
    unique_values = a[col].unique()
    print(f"{col}: {unique_values}")

dribble_outcome: [nan 'Complete' 'Incomplete']
pass_outcome: [nan 'Incomplete' 'Pass Offside' 'Out' 'Unknown' 'Injury Clearance']
shot_outcome: [nan 'Goal' 'Saved' 'Blocked' 'Post' 'Off T' 'Wayward' 'Saved to Post'
 'Saved Off Target']
duel_outcome: [nan 'Success In Play' 'Won' 'Lost In Play' 'Lost Out' 'Success Out']
interception_outcome: [nan 'Lost In Play' 'Lost Out' 'Success In Play' 'Won' 'Success Out']


In [11]:
# Get maximum count for each event type across all players
max_vals_for_normalization = a.groupby(['player', 'type']).size().reset_index(name='count')
max_vals_for_normalization = max_vals_for_normalization.groupby('type')['count'].max().to_dict()

print("Maximum values for each event type:")
for event_type, max_count in sorted(max_vals_for_normalization.items()):
  print(f"{event_type}: {max_count}")


Maximum values for each event type:
50/50: 5
Bad Behaviour: 1
Ball Receipt*: 506
Ball Recovery: 40
Block: 21
Carry: 459
Clearance: 32
Dispossessed: 16
Dribble: 32
Dribbled Past: 9
Duel: 31
Error: 2
Foul Committed: 15
Foul Won: 19
Goal Keeper: 119
Injury Stoppage: 7
Interception: 12
Miscontrol: 22
Offside: 2
Own Goal Against: 1
Pass: 562
Player Off: 3
Player On: 3
Pressure: 156
Shield: 3
Shot: 25
Substitution: 6


In [ ]:
max_vals

In [12]:
max_vals_for_normalization

{'50/50': 5,
 'Bad Behaviour': 1,
 'Ball Receipt*': 506,
 'Ball Recovery': 40,
 'Block': 21,
 'Carry': 459,
 'Clearance': 32,
 'Dispossessed': 16,
 'Dribble': 32,
 'Dribbled Past': 9,
 'Duel': 31,
 'Error': 2,
 'Foul Committed': 15,
 'Foul Won': 19,
 'Goal Keeper': 119,
 'Injury Stoppage': 7,
 'Interception': 12,
 'Miscontrol': 22,
 'Offside': 2,
 'Own Goal Against': 1,
 'Pass': 562,
 'Player Off': 3,
 'Player On': 3,
 'Pressure': 156,
 'Shield': 3,
 'Shot': 25,
 'Substitution': 6}